In [1]:
import pandas as pd 
from datetime import datetime

#  CONFIG 
filename = "Spring 2025 CoT HS Administration respondent actions - Spring 2025 CoT HS Administration respondent actions.csv"
time_format = "%m/%d/%Y %H:%M:%S"  # adjust if needed

# LOAD CSV INTO DATAFRAME 
df = pd.read_csv(filename)

# Merge Date + Time into one column
df["DateTime"] = pd.to_datetime(df["Date"] + " " + df["Time"], format=time_format)

# Identify pause users 
pause_flags = df.groupby(["Assignment","Activities"])["Action"].apply(
    lambda x: any("Pause activity" in a for a in x)
).reset_index(name="HasPause")

# Get start/end times 
def get_times(group):
    start = group.loc[group["Action"].str.contains("Begin activity"), "DateTime"].min()
    end = group.loc[group["Action"].str.contains("End activity"), "DateTime"].max()
    return pd.Series({"Start": start, "End": end})

times = df.groupby(["Assignment","Activities"]).apply(get_times).reset_index()
times["Duration"] = times["End"] - times["Start"]

#  Keep only same-day exams 
times = times[times["Start"].dt.date == times["End"].dt.date]

# Merge and subset no-pause
merged = times.merge(pause_flags, on=["Assignment","Activities"])
no_pause = merged[merged["HasPause"] == False]

# Compute summary stats 
def duration_summary(x):
    return pd.Series({
        "Mean": x.mean(),
        "SD": x.std(),
        "Min": x.min(),
        "Max": x.max(),
        "Median": x.median(),
        "n": x.count(),
        "P25": x.quantile(0.25),
        "P75": x.quantile(0.75),
        "P90": x.quantile(0.90)
    })

summary = (
    no_pause.groupby(["Activities"])["Duration"]
    .apply(duration_summary)
    .reset_index()
)

# --- Save to CSV ---
output_file = "no_pause_summary_full.csv"
summary.to_csv(output_file, index=False)

print(f"✅ Saved full summary with percentiles to {output_file}")

✅ Saved full summary with percentiles to no_pause_summary_full.csv


In [2]:
import pandas as pd

# Load the existing long summary
df = pd.read_csv("no_pause_summary_full.csv")

# Pivot from long → wide
wide = df.pivot(index="Activities", columns="level_1", values="Duration").reset_index()

# Clean up column names
wide.columns.name = None
wide.columns = ["Activities"] + list(wide.columns[1:])

# Remove '0 days' and 'days' text
for col in wide.columns:
    if col != "Activities":
        wide[col] = wide[col].astype(str).str.replace("0 days ", "").str.replace("days", "").str.strip()

# Save to a new CSV
output_file = "no_pause_summary_horizontal.csv"
wide.to_csv(output_file, index=False)

# Print preview
print("\n🕒 Horizontal summary table created!\n")
print(wide.to_string(index=False))
print(f"\n✅ Saved to {output_file}")



🕒 Horizontal summary table created!

         Activities      Max               Mean          Median      Min             P25             P75             P90                 SD  n
 Spr 25 CoT Form A1 02:23:40 00:31:11.111111111        00:26:16 00:02:17 00:08:20.750000        00:37:59 01:10:23.500000 00:30:55.257098514 36
Spr 25 CoT Form A1r 01:50:54 00:34:43.678571428 00:36:34.500000 00:05:59 00:15:37.500000 00:46:20.750000 00:55:55.600000 00:22:43.398532903 28
 Spr 25 CoT Form B1 03:49:04 00:28:48.431818181        00:22:58 00:05:26 00:09:25.750000 00:39:16.250000 00:42:15.600000 00:34:06.999226295 44
Spr 25 CoT Form B1r 04:20:19    00:32:47.175000        00:24:46 00:03:50 00:06:49.250000        00:38:43 01:03:11.100000 00:42:26.881732301 40

✅ Saved to no_pause_summary_horizontal.csv
